In [ ]:
import os
import math
import wandb
import random
import logging
import inspect
import argparse
import datetime
import subprocess

from pathlib import Path
from tqdm.auto import tqdm
from einops import rearrange
from omegaconf import OmegaConf
from safetensors import safe_open
from typing import Dict, Optional, Tuple
from PIL import Image

import torch
import torchvision
import torch.nn.functional as F
import torch.distributed as dist
from torch.optim.swa_utils import AveragedModel
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
import numpy as np

import diffusers
from diffusers import AutoencoderKL, DDIMScheduler
from diffusers.models import UNet2DConditionModel
from diffusers.pipelines import StableDiffusionPipeline
from diffusers.optimization import get_scheduler
from diffusers.utils import check_min_version
from diffusers.utils.import_utils import is_xformers_available

import transformers
from transformers import CLIPTextModel, CLIPTokenizer, CLIPVisionModelWithProjection, CLIPImageProcessor
# from ip_adapter import IPAdapterFull

# from animatediff.data.dataset import WebVid10M, PexelsDataset
from animatediff.utils.util import save_videos_grid, pad_image
from accelerate import Accelerator
from einops import repeat
from animate import MagicAnimate
from animatediff.magic_animate.controlnet import ControlNetModel
import importlib
from animatediff.data.dataset import WebVid10M, PexelsDataset
import webdataset as wds
from animatediff.data.dataset_wds import S3VideosIterableDataset

/data/work/animate_based_ap_ctrl/animatediff/magic_animate/pipeline.py:44: FutureWarning: Importing `DiffusionPipeline` or `ImagePipelineOutput` from diffusers.pipeline_utils is deprecated. Please import from diffusers.pipelines.pipeline_utils instead.
  from diffusers.pipeline_utils import DiffusionPipeline
/home/juneliang/.local/lib/python3.8/site-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/home/juneliang/.local/lib/python3.8/site-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_11m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts wit

In [2]:
config_path = "/data/work/animate_based_ap_ctrl/configs/training/train_stage1_w_imageencoder_celebv_nocenterface_cfg_train_zeroembed_plusbank.yaml"
config = OmegaConf.load(config_path)

In [4]:
eval_dataset = S3VideosIterableDataset(**config.validation_data['dataset'])

In [9]:
eval_dataloader = wds.WebLoader(
            eval_dataset, 
            batch_size=1,
            shuffle=False,
            num_workers=8,
            collate_fn = None,
        ).with_length(len(eval_dataset))

In [10]:
eval_dataloader[0]

NotImplementedError: 

In [ ]:
for idx,data in tqdm(enumerate(eval_dataset)):
    print(data['pixel_values'].shape)
    break

0it [00:00, ?it/s]

{'pixel_values': tensor([[[[-0.2477, -0.2639, -0.2729,  ...,  0.0520,  0.0586,  0.0601],
          [-0.2283, -0.2432, -0.2541,  ...,  0.0510,  0.0568,  0.0694],
          [-0.1867, -0.2030, -0.2243,  ...,  0.0529,  0.0577,  0.0651],
          ...,
          [ 0.3725,  0.3725,  0.3725,  ...,  0.3950,  0.3941,  0.3920],
          [ 0.3725,  0.3725,  0.3725,  ...,  0.3950,  0.3941,  0.3889],
          [ 0.3725,  0.3725,  0.3725,  ...,  0.3950,  0.3941,  0.3889]],

         [[ 0.1693,  0.1675,  0.1652,  ...,  0.3187,  0.3253,  0.3267],
          [ 0.1848,  0.1862,  0.1820,  ...,  0.3176,  0.3235,  0.3361],
          [ 0.2153,  0.2205,  0.2060,  ...,  0.3196,  0.3243,  0.3317],
          ...,
          [ 0.5890,  0.5922,  0.5922,  ...,  0.4902,  0.4960,  0.4940],
          [ 0.5922,  0.5922,  0.5922,  ...,  0.4902,  0.4960,  0.4909],
          [ 0.5922,  0.5922,  0.5922,  ...,  0.4902,  0.4960,  0.4909]],

         [[ 0.4438,  0.4420,  0.4364,  ...,  0.4285,  0.4351,  0.4365],
          [ 0

/home/juneliang/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
